In [23]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/test.csv
/kaggle/input/train.csv


In [24]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score, learning_curve, train_test_split
from sklearn.metrics import precision_score, roc_auc_score, recall_score, confusion_matrix, roc_curve, precision_recall_curve, accuracy_score
import xgboost as xgb
import warnings

In [25]:
train_df=pd.read_csv('/kaggle/input/train.csv')
test_data=pd.read_csv('/kaggle/input/test.csv')
y=train_df['Attrition']
b=list(train_df.columns)

In [26]:
b.remove('Attrition')
train_data=train_df[b]

In [27]:
def data_clean(df1):
    X=df1.copy()
    del(X['Id'])
    
    # convert age to groups
    X['AgeGroup']=X['Age']
    X.loc[(X.Age <= 20),  'AgeGroup'] = 'New'
    X.loc[((X.Age > 20) & (X.Age<=30)),  'AgeGroup'] = 'Young'
    X.loc[((X.Age > 30) & (X.Age<=40)),  'AgeGroup'] = 'Experienced'
    X.loc[((X.Age > 40) & (X.Age<=50)),  'AgeGroup'] = 'Senior'
    X.loc[(X.Age > 50),  'AgeGroup'] = 'Old'
    del(X['Age'])
    
    return X

In [28]:
df=data_clean(train_data)
from sklearn import preprocessing
encoder_b=preprocessing.LabelEncoder().fit(df['BusinessTravel'])
encoder_d=preprocessing.LabelEncoder().fit(df['Department'])
encoder_e=preprocessing.LabelEncoder().fit(df['EducationField'])
encoder_g=preprocessing.LabelEncoder().fit(df['Gender'])
encoder_j=preprocessing.LabelEncoder().fit(df['JobRole'])
encoder_m=preprocessing.LabelEncoder().fit(df['MaritalStatus'])
encoder_o=preprocessing.LabelEncoder().fit(df['OverTime'])
encoder_a=preprocessing.LabelEncoder().fit(df['AgeGroup'])

In [29]:
from sklearn.preprocessing import StandardScaler

In [48]:
def encoded(X):    
    # label encoding for categorical featueres
    from sklearn import preprocessing
    X_cat=X.select_dtypes(include=[object])
    encoders=[encoder_b,encoder_d,encoder_e,encoder_g,encoder_j,encoder_m,encoder_o,encoder_a]
    cols=list(X_cat.columns)
    df2=X_cat.copy()
    for i in range(0,8):
        col=cols[i]
        enc=encoders[i]
        df2[col]=enc.transform(df2[col])
    X_cat_encod=df2
    X_numeric=X.select_dtypes(include=['int64','float64'])
    std = StandardScaler()
    #scaled = std.fit_transform(X_numeric)
    #X_num= pd.DataFrame(scaled,columns=X_numeric.columns)
    X_num=X_numeric
    X_final=X_cat_encod.copy()
    X_final[X_num.columns]=X_num[X_num.columns]
    scaled = std.fit_transform(X_final)
    return scaled

In [49]:
def final_X(data):
    X=data_clean(data)
    X2=encoded(X)
    return X2

In [50]:
train_X=final_X(train_data)
test_X=final_X(test_data)

In [52]:
test_X

array([[ 0.59014837, -0.49436746,  0.55219089, ..., -0.07481551,
         1.38047651,  0.        ],
       [ 0.59014837,  1.44190509,  0.55219089, ...,  0.78282572,
        -0.03616969,  0.        ],
       [ 0.59014837, -0.49436746,  0.55219089, ...,  0.78282572,
         0.67215341,  0.        ],
       ...,
       [ 0.59014837, -0.49436746,  0.55219089, ..., -0.07481551,
        -1.45281589,  0.        ],
       [ 0.59014837, -0.49436746,  0.55219089, ...,  1.06870613,
        -0.03616969,  0.        ],
       [ 0.59014837, -0.49436746, -0.90584124, ..., -0.64657633,
         1.38047651,  0.        ]])

In [33]:
def scoring():
    clf,X_test,y_test=classifier
    from sklearn.metrics import roc_auc_score
    y_pred=clf.predict(X_test)
    score=roc_auc_score(y_test,y_pred)
    
    return score

In [34]:
def output_file():
    clf=classifier[0]
    test=test_X
    attrition_prob=clf.predict_proba(test)
    ans=pd.DataFrame(test_data['Id'])
    ans['Attrition']=attrition_prob[:,1]
    
    return ans

In [104]:
from sklearn.ensemble import GradientBoostingClassifier
GradientBoostingClassifier?


In [163]:
def gradient_boosted_model():
    from sklearn.model_selection import train_test_split
    X_train,X_test,y_train,y_test=train_test_split(train_X,y)
    
    ## model import
    from sklearn.ensemble import GradientBoostingClassifier
    clf=GradientBoostingClassifier(criterion= 'friedman_mse', learning_rate=0.2, loss= 'deviance',
                                   max_depth= 3, max_features= 'sqrt',min_samples_leaf= 0.1,
                                   min_samples_split= 0.13636363636363638, n_estimators= 100, 
                                   subsample=0.95).fit(train_X,y)
    ## 
    return clf,X_test,y_test

In [120]:
def random_forest():
    from sklearn.model_selection import train_test_split
    X_train,X_test,y_train,y_test=train_test_split(train_X,y)
    
    ## model import
    from sklearn.ensemble import RandomForestClassifier
    clf= RandomForestClassifier(max_depth=10,n_estimators=500,oob_score=True,min_samples_split=5,random_state=0,
                             min_samples_leaf=2,criterion='gini',max_features=20).fit(X_train,y_train)
    ## 
    return clf,X_test,y_test

In [121]:
def neural_network():
    from sklearn.model_selection import train_test_split
    X_train,X_test,y_train,y_test=train_test_split(train_X,y)
    
    ## model import
    from sklearn.neural_network import MLPClassifier
    clf= MLPClassifier(hidden_layer_sizes=(27,27,27), activation='relu', solver='adam',
                       max_iter=10000).fit(X_train,y_train)
    ## 
    return clf,X_test,y_test

In [122]:
def svm_classifier():
    from sklearn.model_selection import train_test_split
    X_train,X_test,y_train,y_test=train_test_split(train_X,y)
    
    ## model import
    from sklearn import svm
    clf= svm.SVC(C=0.5,kernel='rbf',degree=3,gamma='scale',coef0=0.0,shrinking=True,probability=True,tol=0.001,
    cache_size=200,verbose=False,max_iter=-1,decision_function_shape='ovr',break_ties=False,random_state=None
                ).fit(X_train,y_train)
    ## 
    return clf,X_test,y_test

In [123]:
from xgboost import XGBClassifier

In [195]:
def xgb_classifier():
    from sklearn.model_selection import train_test_split
    X_train,X_test,y_train,y_test=train_test_split(train_X,y)
    
    ## model import
    clf=XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.6, gamma=0.1, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.01, max_delta_step=0, max_depth=8,
              min_child_weight=1,
              n_estimators=300, n_jobs=-1, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1.0, tree_method=None,
              validate_parameters=False, verbosity=None).fit(X_train,y_train)
    ## 
    return clf,X_test,y_test

In [41]:
train_X.columns

Index(['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole',
       'MaritalStatus', 'OverTime', 'AgeGroup', 'DistanceFromHome',
       'Education', 'EmployeeNumber', 'EnvironmentSatisfaction',
       'JobInvolvement', 'JobSatisfaction', 'MonthlyIncome',
       'NumCompaniesWorked', 'PercentSalaryHike', 'PerformanceRating',
       'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager', 'CommunicationSkill', 'Behaviour'],
      dtype='object')

In [42]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [95]:
from sklearn.ensemble import GradientBoostingClassifier
GradientBoostingClassifier?

In [103]:
def best_param_gridcv():
    parameters = {
    "loss":["deviance"],
    "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    "min_samples_split": np.linspace(0.1, 0.5, 12),
    "min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "max_depth":[3,5,8],
    "max_features":["log2","sqrt"],
    "criterion": ["friedman_mse",  "mae"],
    "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators":[10]
       }

    clf = GridSearchCV(GradientBoostingClassifier(), parameters, cv=10, n_jobs=-1)
    
    start_time = timer(None) # timing starts from this point for "start_time" variable
    clf.fit(train_X,y)
    timer(start_time) # timing ends here for "start_time" variable
    print(clf.score(train_X,y))
    print(clf.best_params_)
    print('\n Best estimator:')
    print(clf.best_estimator_)
    print('\n Best accuracy for %d-fold search with %d parameter combinations:' % (folds, param_comb))
    print(clf.best_score_ )
    print('\n Best hyperparameters:')
    print(clf.best_params_)


In [98]:
def best_parameters():
    cfl = XGBClassifier(n_jobs = -1)
    


# A parameter grid for XGBoost
    params = {
                  'n_estimators' : [100, 200, 500, 750],
                  'learning_rate' : [0.01, 0.02, 0.05, 0.1, 0.25],
                  'subsample': [0.6, 0.8, 1.0],
                  'max_depth': [3, 4, 5, 10, 12],
                   
                  'min_child_weight': [1, 5, 7, 10],
                  'gamma': [0.1, 0.5, 1, 1.5, 5],
                  'colsample_bytree': [0.6, 0.8, 1.0]
                   }
                  

    folds = 5
    param_comb = 800

    random_search = RandomizedSearchCV(cfl, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=-1, cv=5, verbose=3, random_state=42)

    # Here we go
    start_time = timer(None) # timing starts from this point for "start_time" variable
    random_search.fit(train_X, y)
    timer(start_time) # timing ends here for "start_time" variable
    print('\n Best estimator:')
    print(random_search.best_estimator_)
    print('\n Best accuracy for %d-fold search with %d parameter combinations:' % (folds, param_comb))
    print(random_search.best_score_ )
    print('\n Best hyperparameters:')
    print(random_search.best_params_)

In [102]:
#best_param_gridcv()


 Time taken: 3 hours 31 minutes and 9.63 seconds.
0.788083538083538
{'criterion': 'friedman_mse', 'learning_rate': 0.2, 'loss': 'deviance', 'max_depth': 8, 'max_features': 'sqrt', 'min_samples_leaf': 0.1, 'min_samples_split': 0.13636363636363638, 'n_estimators': 10, 'subsample': 0.95}

 Best estimator:


NameError: name 'random_search' is not defined

In [200]:
classifier=xgb_classifier()

In [201]:
scoring()

0.9592760180995474

In [202]:
output_file().to_csv('submission9.csv',index=False)